In [1]:
import gc
import torch

def cleanup_globals(vars_to_keep: list):
    global_vars = list(globals().keys())
    protected_vars = ['In', 'Out', 'get_ipython', 'exit', 'quit', 'gc', 'torch', 'cleanup_globals']

    for var in global_vars:
        if var not in vars_to_keep and not var.startswith('_') and var not in protected_vars:
            try:
                del globals()[var]
                print(f"{var} 삭제됨")
            except:
                continue
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
# 데이터 셋 불러오기
import boto3
import pandas as pd
import io
from datasets import Dataset

# AWS S3 에 업로드 했던 데이터셋 불러오기
# 엑세스키가 필요함
try:
    credentials_df = pd.read_csv('./ganghyun-dev_accessKeys.csv')

    if not credentials_df.empty:
        aws_access_key_id = credentials_df['Access key ID'].iloc[0].strip()
        aws_secret_access_key = credentials_df['Secret access key'].iloc[0].strip()
    else:
        print("Error: 'aws_credentials.csv' is empty.")
        exit()

except FileNotFoundError:
    print("Error: 'aws_credentials.csv' not found in Drive.")
    print("Please create a file named 'aws_credentials.csv' in your Google Drive with your AWS credentials.")
    exit()
except KeyError:
    print("Error: 'Access key ID' or 'Secret access key' column not found in 'aws_credentials.csv'.")
    print("Please ensure your CSV file has these columns.")
    exit()
except Exception as e:
    print(f"Error loading AWS credentials from CSV: {e}")
    exit()

bucket_name = "dr.hong-s3"

file_key = "dataset/template_generation_dataset_with_temp_policy.xlsx"

s3_client = boto3.client('s3',
                         aws_access_key_id=aws_access_key_id,
                         aws_secret_access_key=aws_secret_access_key)

try:
    file_content = s3_client.get_object(Bucket=bucket_name, Key=file_key)['Body'].read()
    print("파일을 성공적으로 메모리로 불러왔습니다.")

except Exception as e:
    print(f"S3에서 파일을 불러오는 중 오류가 발생했습니다: {e}")
    exit()

# 엑셀 파일을 pandas datafrome 으로 변환
print("파일을 pandas dataframe 로 변환")
df = pd.read_excel(io.BytesIO(file_content))

print("원본 데이터 상위 5개")
print(df.head)
print()

print("Hugging Face Dataset 으로 변환")
gen_dataset = Dataset.from_pandas(df)

gen_train_test_dataset = gen_dataset.train_test_split(test_size=0.2, shuffle=True)
gen_train_dataset = gen_train_test_dataset["train"]
gen_test_dataset = gen_train_test_dataset["test"]

print("최종 분할된 데이터 셋")
print(gen_train_dataset)
print(gen_test_dataset)

cleanup_globals(["gen_train_dataset", "gen_test_dataset"])

파일을 성공적으로 메모리로 불러왔습니다.
파일을 pandas dataframe 로 변환
원본 데이터 상위 5개
<bound method NDFrame.head of                                                template  \
0     {"title": "회사소개서 발송", "text": "안녕하세요 #{수신자명}님,...   
1     {"title": "서비스 소개서 발송", "text": "안녕하세요 #{수신자명}...   
2     {"title": "(전용) 강의 일정 안내 / 화케터", "text": "안녕하세...   
3     {"title": "(공용) 후기 작성 요청_이미지형_01", "text": "[템...   
4     {"title": "(공용) 인보이스 알림_이미지형_01", "text": "■ #...   
...                                                 ...   
1167  {"title": "재입고 알림", "text": "안녕하세요 #{수신자명}님,\n...   
1168  {"title": "사전 구매 예약", "text": "안녕하세요 #{수신자명}님,...   
1169  {"title": "정기구독 결제 안내", "text": "안녕하세요 #{수신자명}...   
1170  {"title": "자동 결제 정보 등록 완료", "text": "안녕하세요 #{수...   
1171  {"title": "자동 결제 안내", "text": "안녕하세요 #{수신자명}님,...   

                                user_input  \
0             안녕하세요, 회사소개서 발송 템플릿 제작 부탁드려요   
1                           서비스 소개서 발송 템플릿   
2        (전용) 강의 일정 안내 / 화케터용 알림톡을 만들고 싶어요   
3             (

In [2]:
import os
import gc
import torch
from huggingface_hub import snapshot_download
from huggingface_hub.utils import RepositoryNotFoundError

def download_model_snapshot(model_id: str, local_dir: str) -> str:
    print(f"'{model_id}' 모델을 '{local_dir}' 경로에 다운로드합니다...")
    try:
        # snapshot_download는 알아서 기존 파일을 체크하고 필요한 것만 다운로드합니다.
        model_path = snapshot_download(
            repo_id=model_id,
            local_dir=local_dir
            # resume_download=True, # 기본값이 True이므로 명시하지 않아도 됨
        )
        print("✅ 모델 준비 완료!")
        return model_path
    except RepositoryNotFoundError:
        print(f"❌ 오류: 모델 ID '{model_id}'를 찾을 수 없습니다.")
        return None
    except Exception as e:
        print(f"❌ 다운로드 중 오류가 발생했습니다: {e}")
        return None
    
# 실행
checkpoint = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
model_path = download_model_snapshot(checkpoint, "./downloaded_model/" + checkpoint.replace("/", "--"))

cleanup_globals(["gen_train_dataset", "gen_test_dataset", "model_path"])

/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'MLP-KTLim/llama-3-Korean-Bllossom-8B' 모델을 './downloaded_model/MLP-KTLim--llama-3-Korean-Bllossom-8B' 경로에 다운로드합니다...


Fetching 12 files: 100%|██████████| 12/12 [00:00<00:00, 15.68it/s]

✅ 모델 준비 완료!


NameError: name 'cleanup_globals' is not defined

In [7]:
# 데이터 셋 전처리
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import torch

# 토크나이저 디스크에서 메모리로 로드
def load_tokenizer_from_local(model_path: str):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return tokenizer

tokenizer = load_tokenizer_from_local(model_path)

# 토큰화 함수 재정의
def tokenize_function(elements):
    # 입력 시퀀스 만들기
    texts = [
        f"{tokenizer.bos_token}user_input: {user_input}\npolicy: {policy}\ntemplate: {template}{tokenizer.eos_token}"
        for user_input, policy, template in zip(elements['user_input'], elements['policy'], elements['template'])
    ]
    # 토큰화 하기
    tokenized = tokenizer(
        texts, 
        truncation=True, 
        max_length=8192,
        padding=False,  # DataCollator 에서 padding 함
    )
    
    tokenized['labels'] = tokenized['input_ids'].copy() # AutoRegressive
    return tokenized

# 데이터셋에 토큰화 함수 적용
print("\nApplying tokenization function to the dataset...")
tokenized_gen_train_datasets = gen_train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=gen_train_dataset.column_names
)
tokenized_gen_eval_datasets = gen_test_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=gen_test_dataset.column_names
)

tokenized_gen_datasets = DatasetDict({
    'train': tokenized_gen_train_datasets,
    'eval': tokenized_gen_eval_datasets
})

print("Tokenized train dataset features:", tokenized_gen_train_datasets.features)
print("\nTokenized test dataset features:", tokenized_gen_eval_datasets.features)
print(f"\nTrain dataset size: {len(tokenized_gen_train_datasets)}")
print(f"Test dataset size: {len(tokenized_gen_eval_datasets)}")

tokenized_path = "./tokenized_datasets/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--")
tokenized_gen_datasets.save_to_disk(tokenized_path)

cleanup_globals(["model_path", "tokenizer", "tokenized_path"])


Applying tokenization function to the dataset...


Map: 100%|██████████| 235/235 [00:00<00:00, 2042.93 examples/s]


Tokenized train dataset features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}

Tokenized test dataset features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}

Train dataset size: 937
Test dataset size: 235


Saving the dataset (1/1 shards): 100%|██████████| 235/235 [00:00<00:00, 1530.62 examples/s]


gen_train_dataset 삭제됨
gen_test_dataset 삭제됨
AutoTokenizer 삭제됨
DataCollatorForSeq2Seq 삭제됨
Dataset 삭제됨
DatasetDict 삭제됨
load_tokenizer_from_local 삭제됨
tokenize_function 삭제됨
tokenized_gen_train_datasets 삭제됨
tokenized_gen_eval_datasets 삭제됨
tokenized_gen_datasets 삭제됨


In [2]:
import torch

# GPU가 사용 가능한지 확인
if torch.cuda.is_available():
    # 현재 사용 중인 메모리 (바이트)
    allocated_bytes = torch.cuda.memory_allocated(device=0)
    # 캐시된 메모리 (바이트)
    reserved_bytes = torch.cuda.memory_reserved(device=0)

    # GB 단위로 변환
    gb_factor = 1024 * 1024 * 1024
    allocated_gb = allocated_bytes / gb_factor
    reserved_gb = reserved_bytes / gb_factor

    print(f"현재 사용 중인 GPU 메모리: {allocated_gb:.2f} GB")
    print(f"현재 캐시된 GPU 메모리: {reserved_gb:.2f} GB")

else:
    print("GPU를 사용할 수 없습니다.")

현재 사용 중인 GPU 메모리: 0.00 GB
현재 캐시된 GPU 메모리: 0.00 GB


In [10]:
cleanup_globals([])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_path = "/mnt/custom-file-systems/s3/shared/downloaded_model/MLP-KTLim--llama-3-Korean-Bllossom-8B"

def load_model_and_tokenizer_from_local(model_path: str):
    # load_in_8bit=True 옵션으로 8비트 양자화를 활성화합니다.
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0, # 양자화에서 제외할 아웃라이어 임계값 설정
        llm_int8_has_fp16_weight=False, # 모델의 일부를 FP16으로 유지할지 설정
        llm_int8_skip_modules=None, # 8비트 양자화에서 제외할 모듈 목록
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=bnb_config,
        attn_implementation="flash_attention_2",
        dtype=torch.float16
    )
    return model, tokenizer

model, tokenizer = load_model_and_tokenizer_from_local(model_path)

# 모델을 kbit 학습을 위해 준비
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
model.config.use_cache = False
model.gradient_checkpointing_enable()

# LoRA 설정
print("LoRA 어댑터 초기화 중...")
lora_config = LoraConfig(
    r=8, # 학습능력치, 높이면 학습 성능이 올라가지만 메모리와 속도에서 손해
    lora_alpha=16, # 가중치 스케일링, 보통 r 의 두배
    target_modules=["q_proj", "v_proj"],  # 타겟 모듈 추가
    # target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # 더 정밀한 파인튜닝을 하려면 타겟 추가
    lora_dropout=0.05, # 과적합이 발생하면 높여볼 수 있음
    bias="none", # none 이 일반적임, 다른 옵션은 영향이 거의 없음
    task_type="CAUSAL_LM", # 모델에 맞춰서
    # inference_mode=False  # 학습 모드 활성화 기본값
)

# 모델을 PEFT 모델로 변환
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

cleanup_globals(["tokenizer", "model"])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LoRA 어댑터 초기화 중...
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424
open 삭제됨
AutoTokenizer 삭제됨
AutoModelForCausalLM 삭제됨
BitsAndBytesConfig 삭제됨
LoraConfig 삭제됨
get_peft_model 삭제됨
prepare_model_for_kbit_training 삭제됨
model_path 삭제됨
load_model_and_tokenizer_from_local 삭제됨
lora_config 삭제됨


In [3]:
from datasets import load_from_disk
from transformers import DataCollatorForSeq2Seq

tokenized_path = "./tokenized_datasets/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--")
tokenized_gen_datasets = load_from_disk(tokenized_path)
tokenized_gen_train_datasets = tokenized_gen_datasets['train']
tokenized_gen_eval_datasets = tokenized_gen_datasets['eval']

# DataCollator 정의 - 패딩과 라벨 처리를 위해 DataCollatorForSeq2Seq 사용
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

In [4]:
import torch
import wandb
import evaluate

from transformers import Trainer, TrainingArguments

# wandb 초기화
wandb.init( # 기록을 위한 정보이며 학습에 영향을 주지 않음
    entity="dr-hong",
    project="dr-hong",
    name="llama-3-Korean-Bllossom-8B-v1",
    config={
        "learning_rate": 1e-5,
        "epochs": 3,
        "batch_size": 4,
        "model_name": "MLP-KTLim/llama-3-Korean-Bllossom-8B",
        "lora_r": 8,
        "lora_alpha": 16,
    }
)

# compute_metrics 함수 정의. llama3 에 적합한 평가지표 선택: ROUGE, BLEU
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
def compute_metrics(eval_preds, compute_result=False):
    predictions, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = predictions.argmax(axis=-1)  # logits → token ids

    labels[labels == -100] = tokenizer.pad_token_id
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_result = rouge.compute(
        predictions=decoded_preds, 
        references=decoded_labels
    )
    
    bleu_result = bleu.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )

    return {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "bleu": bleu_result["bleu"]
    }

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir="./results/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--"),
    # overwrite_output_dir=True, # False 기본값이면 자동으로 중단되었던 학습을 이어서 할 수 있음, True 면 처음부터 다시
    do_train=True, # 학습 단계를 건너뛸 수도 있음
    do_eval=True, # 평가를 따로 진행할 수 있음
    # do_predict=False, # 추론단계는 기본적으로 안함
    eval_strategy="steps", # 일정 step 마다 평가
    # prediction_loss_only=False, # True 면 평가와 추론 작업에서 loss 만 반환함
    per_device_train_batch_size=4, # 학습 배치 사이즈, 기본은 8인데 메모리 절약을 위해 줄임
    per_device_eval_batch_size=4, # 평가 배치 사이즈, 기본은 8인데 메모리 절약을 위해 줄임
    gradient_accumulation_steps=8, # 줄인 배치사이즈를 보완하는 배수, 실제 효과는 배치사이즈와 이 값을 곱한 크기를 배치사이즈로 하는 것과 동일함
    eval_accumulation_steps=64, # 평가 단계에서 GPU 메모리 문제가 발생하면 설정, 키우면 CPU 메모리로 더 자주 결과를 옮김
    eval_delay=30, # 학습 초기 단계에서 평가를 건너뜀, 의미없는 계산을 줄임. 학습셋 937개 / (실질적 배치 사이즈 4 * 8) ~= 30 -> 1epoch 정도는 평가를 건너뜀
    torch_empty_cache_steps=30, # GPU 메모리 정리 함수를 호출하는 시점. 안전하게 eval_step 과 동일한 값으로
    learning_rate=2e-4, # AdamW 옵티마이저를 위한 초기 학습률. LoRA 를 쓴다면 2e-4 가 좋음
    weight_decay=0.01, # 정규화 기능. 일반적으로 0.01 사용. 과적합이면 높임
    # adam_beta1, adam_beta2, adam_epsilon, max_grad_norm 은 기본값으로 사용
    # num_train_epochs=3, # 학습 횟수. 기본 3
    # max_steps 사용 안함
    # lr_scheduler_type, lr_scheduler_kwargs 는 기본값 사용
    warmup_ratio=0.1, # 10% 를 웜업 단계로 설정
    # warmup_step=9 # 3 에포크 * 30 스텝 * 0.1 = 9
    # log_level="passive", # 로그 레벨 기본값 사용. warning
    # log_level_replica, log_on_each_node 는 분산 학습용
    logging_dir="./logs/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--"), # TensorBoard 로 로그 확인 가능
    logging_strategy="steps", # epoch 가 얼마 없어서 steps 사용
    # logging_first_step=False
    logging_steps=5, # 1 epoch 와 동일하게
    # logging_nan_inf_filter=True # 기본 True. nan, inf 를 무시하고 로그를 깔끔하게
    save_strategy="steps", # logging_strategy 와 동일하게
    save_steps=15, # epoch 전략이면 무시됨
    save_total_limit=1, # 저장되는 체크포인트 한개만 유지
    # save_safetensors=True # 기본 True. sagetensor 형식으로 저장
    # save_on_each_node 는 분산 학습에서 사용
    # save_only_model=False # 이어서 학습이 가능하도록 기본값 False 로 유지
    # restore_callback_states_from_checkpoint 은 콜백 기능을 사용하면 설정 가능
    # use_cpu=False # GPU 사용하므로 기본값 False 유지
    # seed, data_seed 는 기본값 사용
    # use_ipex=False # NVIDIA GPU 를 사용하므로 기본값 False 사용
    bf16=True, # 최신 GPU 라면 bf16, 구형이라면 fp16 사용
    # half_precision_backend="auto" # 혼합 정밀도 기능을 위한 라이브러리, 기본값 auto 사용
    tf32=True, # ml.g5.xlarge 인스턴스에서 작업하므로 사용가능한 옵션
    # local_rank=-1 # 분산 학습이 아니므로 기본값 -1
    # ddp_backend="nccl" # 분산 학습이면 사용
    eval_steps=5, # eval_strategy 가 steps 면 logging_steps 와 동일한 값이 기본
    dataloader_num_workers=4, # ml.g5.xlarge 는 cpu 코어수가 4개. 데이터로더가 병렬처리 가능
    disable_tqdm=False, # 진행도 시각화 기능 사용
    # remove_unused_columns=True, # 메모리 절약을 위해 기본값 True 사용
    # label_names 는 label 을 사용하는 것이 기본이고 그렇게 전처리 했음
    load_best_model_at_end=True, # save_total_limit=1 과 조합이 좋음
    metric_for_best_model="eval_loss", # 가장 좋은 모델 선택 기준: eval loss
    greater_is_better=False, # loss 는 낮은게 좋은 결과임
    # ignore_data_skip=False, # 이어서 학습 하기위해 기본값 False 사용
    # fsdp, fsdp_config, accelerator_config, parallelism_config 는 분산 학습에서 사용
    # label_smoothing_factor=0.1 # 과적합이 발생하면 설정
    # optim, optim_args 는 기본인 AdamW 를 사용
    group_by_length=True, # Dynamic Padding 을 했다면 메모리 절약 가능
    report_to="all", # 모든 사용 가능한 플랫폼에 보고, 지금은 wandb
    # ddp_find_unused_parameters, ddp_bucket_cap_mb, ddp_broadcast_buffers 는 분산 환경에서 사용
    # dataloader_pin_memory=True, # GPU -> CPU 메모리 이동이 빨라짐, 기본값 True 사용
    dataloader_persistent_workers=True, # CPU RAM 이 부족하면 False 로
    # dataloader_prefetch_factor=2, # CPU 메모리가 여유롭다면 2 이상으로 사용 가능
    # skip_memory_metrics=True # 메모리 문제의 원인을 찾아야 한다면 False 사용
    # push_to_hub=False # 업로드 안함
    resume_from_checkpoint="./results/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--"), # 학습 재개 가능
    gradient_checkpointing=True, # 메모리 문제 방지
    # eval_do_concat_batches=True, # 평가지표 계산 속도를 위해 기본값 True 사용
    auto_find_batch_size=True, # 다른 배치 사이즈 설정을 덮어씀. 자동으로 실행가능한 최적의 배치 크기를 찾아줌. 처음 한번은 이 기능을 사용해서 배치 크기를 찾는게 좋음
    # full_determinism=False, # 난수 고정 기능. 연구에 사용 가능
    torchdynamo="inductor", # 학습 속도 올려주는 PyTorch 기본 라이브러리
    torch_compile=True, # 모델 최적화 기능
    torch_compile_backend="inductor", # 컴파일 해주는 백엔드 지정
    torch_compile_mode="default",
    neftune_noise_alpha=5.0, # llama 3 같은 명령어 파인튜닝에 효과적인 기법
    batch_eval_metrics=True, # 평가단계 메모리 절약
    # eval_on_start=True, # 평가단계 디버깅을 빠르게 해볼려면 사용
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_gen_train_datasets,
    eval_dataset=tokenized_gen_eval_datasets,
    processing_class=tokenizer,
    # model_init 는 하이퍼 파라미터 튜닝할 때 사용
    compute_metrics=compute_metrics,
    # optimizers 는 기본 AdamW 사용
)

try:
    train_output = trainer.train()
    
    eval_results = trainer.evaluate()
    
except Exception as e:
    print(f"학습 중 오류 발생: {str(e)}")
    raise e

# 최종 모델 저장
model.save_pretrained("./finetuned_model/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--"))

# wandb 종료
wandb.finish()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bill291104 (dr-hong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/transformers/training_args.py:1818: FutureWarning: `torchdynamo` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `torch_compile_backend` instead
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128001}.
/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.
/home/sagemaker-user/.c

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu
30,0.851100,0.712596,0.754690,0.691437,0.754690,0.433456
35,0.686000,0.612044,0.915666,0.883394,0.915666,0.592565
40,0.628600,0.549853,0.888473,0.857504,0.888473,0.677233
45,0.567500,0.512030,0.908745,0.866209,0.908745,0.731451
50,0.482800,0.486619,0.928802,0.874780,0.928802,0.619834
55,0.514900,0.468702,0.898214,0.853021,0.898214,0.629576
60,0.438300,0.457031,0.884503,0.776887,0.884503,0.743210
65,0.477500,0.448802,0.916825,0.872147,0.916825,0.709512
70,0.443300,0.441495,0.942280,0.912324,0.942280,0.764197
75,0.442300,0.436455,0.931395,0.866178,0.931395,0.664749


/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/sagemaker-user/.cache/pypoetry/virtualenvs/dr-hong-pr-60wuyk-o-py3.11/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to 

eval/bleu,▁▄▅▆▄▅▇▆▇▅▆▇█▆
eval/loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁
eval/rouge1,▁▇▆▇▇▆▆▇██▆▆▅▇
eval/rouge2,▁▇▆▇▇▆▄▇█▇▇▆▄▇
eval/rougeL,▁▇▆▇▇▆▆▇██▆▆▄▇
eval/runtime,▂▁▁▁▁▁▁▁▂▁▁▁▁█
eval/samples_per_second,▇███████▇████▁
eval/steps_per_second,▇███████▇████▁
train/epoch,▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇██████
train/global_step,▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇██████
+3,...


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import gc

# 메모리 정리
torch.cuda.empty_cache()
gc.collect()

# 베이스 모델과 토크나이저 로드
base_model_path = "/mnt/custom-file-systems/s3/shared/downloaded_model/MLP-KTLim--llama-3-Korean-Bllossom-8B"
lora_adapter_path = "./finetuned_model/" + "MLP-KTLim/llama-3-Korean-Bllossom-8B".replace("/", "--")

# 8비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

# 추론용 모델 로드 (메모리 효율적 버전)
def load_inference_model_and_tokenizer(base_model_path, lora_adapter_path):
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=bnb_config,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(base_model, lora_adapter_path)
    return model, tokenizer

base_model, tokenizer = load_inference_model_and_tokenizer(base_model_path, lora_adapter_path)

# 추론 함수
def generate_template(user_input, policy):
    prompt = f"""사용자 요청: {user_input}
정책: {policy}
템플릿:"""
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(base_model.device)
    
    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("템플릿:")[-1].strip()

# 사용 예시
user_request = "회사소개서 발송 템플릿 제작 부탁드려요"
policy_text = "정보성 메시지란 정보통신망법 안내서에 '영리목적 광고성 정보의 예외'에 해당하는 메시지입니다."

result = generate_template(user_request, policy_text)
print(result)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{"title": "회사소개서 발송", "text": "안녕하세요 #{수신자명}님,\n\n#{회사명}은 #{업종} 분야에서 활동하는 #{회사명}입니다.\n\n▶ 회사명 : #{회사명}\n▶ 업종 : #{업종}\n▶ 연락처 : #{연락처}\n\n감사합니다.", "button_name": "자세히 보기"}목적: 고객에게 정보를 제공하는 것이 목적이며, 알림톡으로 발송 가능한 정보성 메시지입니다. 정보통신망법과 카카오톡 내부 기준에 따라 심사 진행되고 승인된 알림톡 템
